In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [13]:

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

PERSIST_DIR = './storage'
if not os.path.exists(PERSIST_DIR):
    # load documents and create index
    documents = SimpleDirectoryReader("Data").load_data()
    index = VectorStoreIndex.from_documents(documents, show_progress = True)
    # store it for consecutive use
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

c:\Users\Admin\miniconda3\envs\env-01\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 3/3 [00:01<00:00,  1.98it/s]


In [19]:

queryEngine = index.as_query_engine()
response = queryEngine.query('Who is Ramya Anand')
print(response)

Ramya Anand is a Full Stack Developer with around 14 years of experience in web application design and development. She is a Microsoft Azure certified Cloud Developer with expertise in C# .NET Core, Angular, and Azure Cloud technologies. Ramya has worked on projects involving cloud migration, Azure functions, and events, as well as implementing Azure load testing modules for performance tuning. She has experience in developing .NET Core Microservices, utilizing various front-end and back-end technologies, and has certifications in Azure AI Fundamentals and Azure Developer Associate.


In [20]:
from llama_index.core.response.pprint_utils import pprint_response
response = queryEngine.query('What skills does Ramya Anand have?')
pprint_response(response,show_source=True)

Final Response: Ramya Anand has skills in C#, Python, .NET Core API,
ASP.NET Web API, ASP.NET MVC, Entity Framework, Angular 13, ngRx,
HTML5, CSS3, SASS, JavaScript, Typescript, JSON, XML, XUnit, NUnit,
Jasmine & Karma (Angular), MS SQL Server, IBM DB2, Git, TFS,
Subversion, VisualSourceSafe, Azure App Services, Azure Functions,
AppInsights, KeyValuts, Storage Accounts, Azure SQL DB, ServiceBus,
Visual Studio 2022, and Visual Studio Code.
______________________________________________________________________
Source Node 1/2
Node ID: da4bd68e-e244-4b19-b10c-473bfc1631a6
Similarity: 0.6165451598687764
Text: Page 1 of 2  Ramya Anand  Full stack Developer
C# .NET Core| Angular | Azure Cloud  .NET Application Developer with
around 14 years of experience in web application design and
development.  Microsoft Azure certiﬁed Cloud Developer with experience
in cloud migration projects. A continuo...
______________________________________________________________________
Source Node 2/2
Node ID: 6

In [33]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings, PromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# create index with Documents + LLM
index2 = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model, llm=Settings.llm
)

# retrieves nodes from vector
customRetriever = VectorIndexRetriever(index= index2, similarity_top_k= 4)
# processes nodes
postProcessor = SimilarityPostprocessor(similarity_cutoff= 0.40)
# prompt template
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str
)
# construct query engine and update prompt template
customQueryEngine = RetrieverQueryEngine(customRetriever, node_postprocessors=[postProcessor])
customQueryEngine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# query the RAG
query_str = 'Who is Ramya Anand? How can she become an AI Engineer?'
response2 = customQueryEngine.query(query_str)

# take a look at the prompt
retrieved_nodes = customRetriever.retrieve(query_str)
context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
print(qa_prompt_tmpl.format(query_str=query_str, context_str=context_str))
print(response2)


Context information is below.
---------------------
Page 1 of 2 
Ramya Anand 
Full stack Developer                                                              C# .NET Core| Angular | Azure Cloud 
.NET Application Developer with around 14 years of experience in web application design and development. 
Microsoft
Azure certiﬁed Cloud Developer with experience in cloud migration projects. A continuous learner and an AI enthusiast 
connect.ramyaanand@outlook.com 
+1 
753 881 6545 
Ottawa, Ontario 
linkedin.com/in/link-ramya-anand 
WORK EXPERIENCE 
Full stack Developer 
Modaxo ( Constellation Software, Canada) 
06/2021 - 04/2024
, 
 
Mobility on Demand 
One Mobility is a cloud based Angular application for
end-to-end transit services that provides more
eﬃcient and predictive processes for better user
experience 
Improved on Schedule editor module that drastically
increase automation on scheduling and dispatching to
achieve operational eﬃciencies and productivity 
Worked on MDT-center module